In [11]:
import coverage as cov
import geopandas as gpd
import plotly.express as px

In [12]:
# tles = cov.gen_sats(sat_nos=[39084, 49260]) # L8 & L9
tles = cov.gen_sats(sat_nos=[48915, 48911]) # YAM-3 & YAM-2

times = cov.gen_times(start_yr=2021, start_mo=11, start_day=20, days=1, step_min=1)
df = cov.prop_orbits(tles, times)
df = cov.add_bearing(df)
dt_df = df.set_index("time")


Satellite(s) Loaded from TLE:
[<EarthSatellite YAM-3 catalog #48915 epoch 2021-11-27 14:32:26 UTC>]
[<EarthSatellite YAM-2 catalog #48911 epoch 2021-11-27 14:46:53 UTC>]
Propogation time: 
 2021-11-20 00:00:00+00:00 
to 
 2021-11-20 23:59:00+00:00
Adding YAM-3
Adding YAM-2


In [13]:
df.head()

,time,satellite,lat,lon,altitude,bearing,sunlit
0,2021-11-20 00:00:00+00:00,YAM-3,0.036457,31.485911,531.528559,348.820102,False
1,2021-11-20 00:01:00+00:00,YAM-3,3.807086,30.739550,530.520897,348.828811,False
2,2021-11-20 00:02:00+00:00,YAM-3,7.578202,29.988714,529.667496,348.820176,False
3,2021-11-20 00:03:00+00:00,YAM-3,11.349099,29.228968,528.969092,348.792937,False
4,2021-11-20 00:04:00+00:00,YAM-3,15.119057,28.455562,528.423730,348.744996,False


In [14]:
# df["altitude"].describe()

In [15]:
fig = px.scatter_geo(dt_df, lon="lon", lat="lat", color="satellite", projection="orthographic")
fig.show()

In [16]:
inst = cov.gen_instrument(name="instrument", fl=100, pitch=0.02, h_pix=3000, v_pix=800, mm=True)
inst

{'name': 'instrument',
 'fl': 100,
 'pitch': 0.02,
 'h_pix': 3000,
 'v_pix': 800,
 'mm': True,
 'half_diag_deg': 17.789393810389726,
 'az1': 75.06858282186245,
 'az2': 284.93141717813756}

In [17]:
df, polys = cov.gen_los_offsets(df.dropna(), inst)

In [18]:
cmap = px.colors.qualitative.Plotly
sats = polys.satellite.unique()
color_key = dict(zip(sats, cmap))

polys['color'] = polys.apply(
    lambda row: color_key[row.satellite], axis=1)

In [19]:
polys = polys.to_crs("ESRI:54009")
polys = polys[polys.area < polys.area.quantile(0.98)]
polys

,geometry,satellite,color
0,"POLYGON ((3293941.869 88804.223, 3001700.476 1...",YAM-3,#636EFA
1,"POLYGON ((3213368.392 554479.524, 2922644.809 ...",YAM-3,#636EFA
2,"POLYGON ((3124838.411 1019363.501, 2835025.824...",YAM-3,#636EFA
3,"POLYGON ((3028505.105 1482595.256, 2738996.504...",YAM-3,#636EFA
4,"POLYGON ((2924512.908 1943310.906, 2634696.106...",YAM-3,#636EFA
...,...,...,...
2873,"POLYGON ((3575763.335 -3949195.706, 3240557.93...",YAM-2,#EF553B
2874,"POLYGON ((3571212.246 -3514397.182, 3243944.26...",YAM-2,#EF553B
2875,"POLYGON ((3557596.005 -3072678.613, 3237323.69...",YAM-2,#EF553B
2876,"POLYGON ((3535017.181 -2624938.917, 3220888.70...",YAM-2,#EF553B


In [20]:
polys.explore(color="color", tooltip="satellite")
# for tomorrow... https://towardsdatascience.com/around-the-world-in-80-lines-crossing-the-antimeridian-with-python-and-shapely-c87c9b6e1513